In [9]:
import sys
sys.path.append('..')
sys.path.append('../..')
import pandas as pd
import numpy as np
import datetime
import os
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import dcube as dc


In [3]:
# df_close = get_single_factor_values('CCB__close',date_s,date_e,None,'ccb')

In [4]:
# df_close = get_single_factor_values('CCB__close',date_s,date_e,None,'ccb')

In [26]:
# -*- coding: utf-8 -*-
import math
from scipy.stats import norm
import numpy as np
import pandas as pd 
from scipy import stats
from scipy.optimize import fsolve
def blsprice(S, K, T, r, sigma):
    '''
    input: price标的资产市场价格，strike执行价格，rate无风险利率，time距离到期时间，volatility标的资产价格波动率
    output: 看涨期权价格，看跌期权价格
    '''
    price, strike, rate, time, volatility = float(S), float(K), float(r), float(T), float(sigma)
    d1 = (np.log(price / strike) + (rate + 0.5 * volatility ** 2) * time) / (volatility * np.sqrt(time))
    d2 = d1 - volatility * np.sqrt(time)
    call = price * stats.norm.cdf(d1, 0.0, 1.0) - strike * np.exp(-rate * time) * stats.norm.cdf(d2, 0.0, 1.0)
    put = strike * np.exp(-rate * time) * stats.norm.cdf(-d2, 0.0, 1.0) - price * stats.norm.cdf(-d1, 0.0, 1.0) 
    return call

def ImpliedVolatitityCallObj(call, S, K, T, r, sigma_est=0.1):
    '''
    input:call看涨期权现在实际价格,price标的资产现在价格，strike标的资产协议价格，rate无风险利率，time标的资产剩余年数，volatility_est波动率优化初始值
    output:该看涨期权的隐含波动率iv
    '''
    def difference(sigma_est, S, K, T, r):
        # 根据参数,使用blsprice计算期权价格
        est_call = blsprice(S, K, T, r, sigma_est)
        return est_call - call
    # 存在一个波动率使得下列等式成立
    iv = fsolve(difference, sigma_est, args=(S, K, T, r))[0]
    return iv #iv隐含波动率

# 定义Black-Scholes模型计算期权价格的函数
def calculate_option_price_BS(S, K, T, r, sigma, option_type):
    
    
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return option_price


def calculate_option_delta_BS(S, K, T, r, sigma, option_type):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    
    if option_type == 'call':
        delta = norm.cdf(d1)
    elif option_type == 'put':
        delta = norm.cdf(d1) - 1
    
    return delta

def calculate_option_gamma_BS(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    gamma = norm.pdf(d1) / (S * sigma * math.sqrt(T))
    return gamma

def calculate_option_vega_BS(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    vega = S * norm.pdf(d1) * math.sqrt(T)
    return vega

def calculate_option_theta_BS(S, K, T, r, sigma, option_type):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    if option_type == 'call':
        theta = -S * norm.pdf(d1) * sigma / (2 * math.sqrt(T)) - r * K * math.exp(-r*T) * norm.cdf(d2)
    elif option_type == 'put':
        theta = -S * norm.pdf(d1) * sigma / (2 * math.sqrt(T)) + r * K * math.exp(-r*T) * norm.cdf(-d2)
    
    return theta

def calculate_option_rho_BS(S, K, T, r, sigma, option_type):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    if option_type == 'call':
        rho = K * T * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        rho = -K * T * math.exp(-r * T) * norm.cdf(-d2)
    
    return rho

# 其他函数
def get_all_trade_days(start_date='20000101'):
    df = pro.trade_cal(exchange='', start_date=pd.to_datetime(start_date).strftime('%Y%m%d'),is_open='1')
    trade_days = df['cal_date'].tolist()
    all_days = pd.to_datetime(trade_days)
    all_days = [date.date() for date in all_days]
    return all_days

def read_single_factor_file_nocache(factor_name: str):
    path = r'C:\Users\nibh\Desktop\FZ\CCB\{}.csv'.format(factor_name)
    df = pd.read_csv(path, index_col=0)
    df.index = [datetime.datetime.strptime(i, '%Y-%m-%d').date() for i in df.index]
    return df

#在指定日期和CCB池中读取单因子值
def get_single_factor_values(factor_name: str,start_date: datetime.date ,end_date: datetime.date,pool: list,cache=False):
    df = read_single_factor_file_nocache(factor_name)
    return df.loc[start_date:end_date, pool]

# #参数定义
file_path = r'C:\Users\nibh\Desktop\FZ\CCB\计算结果.xlsx'
option_type = 'call'
r = 0.02    #无风险利率

all_days = get_all_trade_days()
date_s = '20160101'
date_e = '20230626'
date_s = pd.to_datetime(date_s).date()
date_e = pd.to_datetime(date_e).date()
date_list = [date for date in all_days if date_s<=date<=date_e]
ccb_code_list = pd.read_csv('C:\\Users\\nibh\\Desktop\\FZ\\CCB\\CCB_stock_code.csv')['ccb_code'].tolist()
factor_list = ['CCB__close','CCB__strbpremium','CCB__conv_price','CCB__conv_ratio','CCB__conv_value','CCB__ptm',\
               'CCB__S__EODP__S__STD240D','CCB__S__EODP__S_DQ_CLOSE']
option_cal_pnl = {}
# option_cal_df = pd.DataFrame()
for f in factor_list:
    df = get_single_factor_values(f,date_s,date_e,ccb_code_list)
    option_cal_pnl[f] = df
option_cal_pnl = pd.Panel(option_cal_pnl)


In [25]:
option_cal_pnl.axes

[Index(['CCB__close', 'CCB__strbpremium', 'CCB__conv_price', 'CCB__conv_ratio',
        'CCB__conv_value', 'CCB__ptm', 'CCB__S__EODP__S__STD240D',
        'CCB__S__EODP__S_DQ_CLOSE'],
       dtype='object'),
 Index([2016-01-04, 2016-01-05, 2016-01-06, 2016-01-07, 2016-01-08, 2016-01-11,
        2016-01-12, 2016-01-13, 2016-01-14, 2016-01-15,
        ...
        2023-06-09, 2023-06-12, 2023-06-13, 2023-06-14, 2023-06-15, 2023-06-16,
        2023-06-19, 2023-06-20, 2023-06-21, 2023-06-26],
       dtype='object', length=1817),
 Index(['128015.SZ', '117128.SZ', '117117.SZ', 'S72466.IOC', '117137.SZ',
        '117164.SZ', '128108.SZ', '123001.SZ', '117127.SZ', '117118.SZ',
        ...
        '123159.SZ', '113525.SH', '113638.SH', '113656.SH', '113580.SH',
        '113625.SH', '113539.SH', '113600.SH', '113043.SH', '113644.SH'],
       dtype='object', length=1384)]

In [28]:
BS_pnl = {}
for date in tqdm(date_list):
    option_cal_df = option_cal_pnl.loc[:,date]
    option_cal_df['CCB__strbpremium_div_conv_ratio'] = option_cal_df['CCB__strbpremium'].div(option_cal_df['CCB__conv_ratio'].replace(0,np.nan))
    BS_df = pd.DataFrame()
#     BS_df['转+A:R债代码'] = option_cal_df['转+A:R债代码']
    BS_df['转股比例'] = option_cal_df['CCB__conv_ratio']
    BS_df['转股价值'] = option_cal_df['CCB__conv_value']
    BS_df['正股近12个月的年化波动率'] = option_cal_df['CCB__S__EODP__S__STD240D']*250**0.5
    # BS_df['BS期权价格_zl'] = option_cal_df.apply(lambda row: blsprice(row['正股价格'],row['转债行权价格'],r,row['自愿转股期限'],row['正股近24个月的波动率']), axis=1)
    # 每份正股对应的BS期权价格和希腊字母
    BS_df['BS期权价格'] = option_cal_df.apply(lambda row: calculate_option_price_BS(row['CCB__S__EODP__S_DQ_CLOSE'],row['CCB__conv_price'],row['CCB__ptm'],r,row['CCB__S__EODP__S__STD240D'],option_type), axis=1)
    BS_df['BS_delta'] = option_cal_df.apply(lambda row: calculate_option_delta_BS(row['CCB__S__EODP__S_DQ_CLOSE'],row['CCB__conv_price'],row['CCB__ptm'],r,row['CCB__S__EODP__S__STD240D'],option_type), axis=1)
    BS_df['BS_gamma'] = option_cal_df.apply(lambda row: calculate_option_gamma_BS(row['CCB__S__EODP__S_DQ_CLOSE'],row['CCB__conv_price'],row['CCB__ptm'],r,row['CCB__S__EODP__S__STD240D']), axis=1)
    BS_df['BS_vega'] = option_cal_df.apply(lambda row: calculate_option_vega_BS(row['CCB__S__EODP__S_DQ_CLOSE'],row['CCB__conv_price'],row['CCB__ptm'],r,row['CCB__S__EODP__S__STD240D']), axis=1)
    BS_df['BS_theta'] = option_cal_df.apply(lambda row: calculate_option_theta_BS(row['CCB__S__EODP__S_DQ_CLOSE'],row['CCB__conv_price'],row['CCB__ptm'],r,row['CCB__S__EODP__S__STD240D'],option_type), axis=1)
    BS_df['BS_rho'] = option_cal_df.apply(lambda row: calculate_option_rho_BS(row['CCB__S__EODP__S_DQ_CLOSE'],row['CCB__conv_price'],row['CCB__ptm'],r,row['CCB__S__EODP__S__STD240D'],option_type), axis=1)
    BS_df['BS_iv'] = option_cal_df.apply(lambda row: ImpliedVolatitityCallObj(row['CCB__strbpremium_div_conv_ratio'],row['CCB__S__EODP__S_DQ_CLOSE'],row['CCB__conv_price'],row['CCB__ptm'],r), axis=1)
    BS_df['BS_iv_bias'] = BS_df['BS_iv'] - BS_df['正股近12个月的年化波动率']
    BS_pnl[date] = BS_df
BS_pnl = pd.Panel(BS_pnl)

100%|████████████████████████████████████████████████████████████████████████████| 1817/1817 [2:54:02<00:00,  5.75s/it]


In [6]:
# (get_single_factor_values('CCB__S__EODP__S_DQ_PCTCHANGE','20210101','20230731',None,'ccb')/100).rolling(500,min_periods=20).std().iloc[-1]*250**0.5

In [7]:
# BS_df['正股近12个月的波动率']*252**0.5

In [29]:
path = r'C:\Users\nibh\Desktop\FZ\CCB'
file_name = '{}\\CCB__BS_iv.csv'.format(path)
BS_pnl.loc[:,:,'BS_iv'].T.to_csv(file_name)

file_name = '{}\\CCB__BS_iv_bias.csv'.format(path)
BS_pnl.loc[:,:,'BS_iv_bias'].T.to_csv(file_name)


In [1]:
import pickle
with open("BS_pnl.pkl", "wb") as f:
    pickle.dump(BS_pnl, f)
    
# 加载本地的 Panel 对象
with open("BS_pnl.pkl", "rb") as f:
    BS_pnl = pickle.load(f)

In [12]:
path = r'C:\Users\nibh\Desktop\FZ\CCB'

file_name = '{}\\CCB__BS_price.csv'.format(path)
BS_pnl.loc[:,:,'BS期权价格'].T.to_csv(file_name)


In [11]:
BS_pnl.axes

[Index([2016-01-04, 2016-01-05, 2016-01-06, 2016-01-07, 2016-01-08, 2016-01-11,
        2016-01-12, 2016-01-13, 2016-01-14, 2016-01-15,
        ...
        2023-06-09, 2023-06-12, 2023-06-13, 2023-06-14, 2023-06-15, 2023-06-16,
        2023-06-19, 2023-06-20, 2023-06-21, 2023-06-26],
       dtype='object', length=1817),
 Index(['128015.SZ', '117128.SZ', '117117.SZ', 'S72466.IOC', '117137.SZ',
        '117164.SZ', '128108.SZ', '123001.SZ', '117127.SZ', '117118.SZ',
        ...
        '123159.SZ', '113525.SH', '113638.SH', '113656.SH', '113580.SH',
        '113625.SH', '113539.SH', '113600.SH', '113043.SH', '113644.SH'],
       dtype='object', length=1384),
 Index(['转股比例', '转股价值', '正股近12个月的年化波动率', 'BS期权价格', 'BS_delta', 'BS_gamma',
        'BS_vega', 'BS_theta', 'BS_rho', 'BS_iv', 'BS_iv_bias'],
       dtype='object')]